<a href="https://colab.research.google.com/github/dmswl0707/Face-Expression-Recognition_with-DNN/blob/main/training_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 train / test / validation

##데이터 다운 및 데이터 로드

1.def load_data():
- transfotmation = transforms.Compose(tranaforms.ToTensor, transforms.Nomalize) 
 - transforms.Compose = transformation으로 구성 묶기
 - transforms.ToTensor = 텐서형태로 변환
 - transforms.Nomalize = 변형 개체 정규화하기

- train/test_dataset=dataset.___('data/', train=True/False, transform=transformation, download=True)
 - train/test = True, False
 - download = True 로 설정

2. def split_train_val_test(train, test, p = train 데이터 비율)
: 전체 데이터셋의 80퍼센트는 train, 20퍼센트는 val (기능 설정)
- train_set_sum=int/(len(train_dataset.data)*p)
 - torch.utils.data.random_split(전체 데이터, train 데이터 갯수, 전체 데이터 - train 데이터 갯수)
 - torch.utils.data.DataLoader(데이터, batch_size=, shuffle=True/False)


    



In [ ]:


# mnist 데이터 다운받고 불러오기
def load_mnist():
    transformation = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])

    train_dataset=datasets.MNIST('data/', train=True, transform=transformation, download=True)
    test_dataset=datasets.MNIST('data/', train=False, transform=transformation, download=True)


    return train_dataset, test_dataset

#불러온 데이터 로드하기
#로드 할때 train 데이터에 20%는 validation을 위해 나
def split_train_val_test(train_dataset, test_dataset, p=0.8):
    train_set_num = int(len(train_dataset.data)*p)

    train_data, val_data = torch.utils.data.random_split(train_dataset, [train_set_num, len(train_dataset)-train_set_num])
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)
    test_loader=torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=True)


    return train_loader, val_loader, test_loader



##train 메소드
1. def train(epoch, model, train_loader, val_loader)
- train_loss, train_acc, global_step = 0.0, 0.0, 0으로 초기화
- for e in range(epoch): 정해진 에폭동안 상수 e를 돌림
 - for batch_idx, (input, target) in enumerate(train_loader):
 batch_idx, (input, target)을 enumerate로 돌림
 - global_step
 - 
 - 
- optimizer.zero_grad() : gradient descent 직전에 초기화
- loss.backward() : 설정한 loss로 부터 역전파를 통해 각 loss에 대한 gradient를 구해준다.
- optomozer.step() : 매개변수 갱신
- with torch.no_grad(): requires_grad=True, True인 모든 작업을 추적
 - val_acc, val_loss = test(model, val_loader) 객체 설정
 - writer.add_scalar('Accuracy/val', val_acc, global_step)
 - writer.add_scalar('Loss/val', val_loss, global_step)
 - print()
- train_loss, train_acc = 0.0, 0.0으로 초기화
- epoch, train_loss, train_acc, val_loss, val_acc이 출력됨


 

In [ ]:
#train 및 test하는 메소
def train(epoch, model, train_loader, val_loader):
    #model.train()

    train_loss = 0.0
    train_acc = 0.0
    global_step = 0
    for e in range(epoch):
        for batch_idx, (input, target) in enumerate(train_loader):
            global_step = e * len(train_loader) * batch_size + (batch_idx * batch_size)

            input, target = input.cuda(), target.cuda()
            input, target = Variable(input), Variable(target)

            optimizer.zero_grad()

            output = model(input)
            loss=F.nll_loss(output, target)
            preds = output.data.max(dim=1, keepdim=True)[1]
            acc=preds.eq(target.data.view_as(preds)).cpu().sum()
            train_loss +=F.nll_loss(output, target, reduction='sum').data.item()
            train_acc +=acc.item()

            loss.backward()
            optimizer.step()

        with torch.no_grad():
            val_acc, val_loss = test(model, val_loader)
            writer.add_scalar('Loss/val', val_loss, global_step)
            writer.add_scalar('Accuracy/val', val_acc, global_step)

        loss = train_loss/len(train_loader.dataset)
        accuracy = 100.0 * train_acc/len(train_loader.dataset)
        writer.add_scalar('Loss/train', loss, global_step)
        writer.add_scalar('Accuracy/train', accuracy, global_step)

        print('Epoch: {0:4} iter: {1:10} loss {2:3.5f} acc {3:3.5f} \t v_loss {4:3.5f} v_acc {5:3.5f}'
              .format(e+1, global_step, loss, accuracy, val_loss, val_acc))
        
        train_loss = 0.0
        train_acc = 0.03



##test 메소드

- running_acc, running_loss = 0.0 으로 초기화
 - 
 - 
 - 
 - 
- accuracy와 loss에 대한 정의
 - accuracy = 100.0 * running_acc / len(data_loader.dataset)
    loss = running_loss / len(data_loader.dataset)
- test_loss, test_acc이 출력됨

In [ ]:
def test(model, data_loader):
    #model.eval()

    running_acc = 0.0
    running_loss = 0.0
    for batch_idx, (input, target) in enumerate(data_loader):

        input, target = input.cuda(), target.cuda()
        input, target = Variable(input), Variable(target)

        output = model(input)

        preds = output.data.max(dim=1, keepdim=True)[1]
        acc = preds.eq(target.data.view_as(preds)).cpu().sum()
        running_loss +=F.nll_loss(output, target, reduction='sum').data.item()
        running_acc += acc.item()

    accuracy = 100.0 * running_acc / len(data_loader.dataset)
    loss = running_loss / len(data_loader.dataset)

    return accuracy, loss



#summary writer

In [ ]:
if __name__=="__main__":
    writer = SummaryWriter()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #hyperparameter
    lr = 0.003
    epoch = 50
    batch_size = 512
    train_val_p=0.8

    #데이터 셋 로드 및 train, val, test로 데이터 셋 나눔
    train_dataset, test_dataset = load_mnist()
    train_loader, val_loader, test_loader = split_train_val_test(train_dataset, test_dataset, p=train_val_p)

    #모델 불러오고, optimizer 설정
    model = Net().to(device)                            # 모델 불러오기
    optimizer = optim.SGD(model.parameters(), lr)      #optimizer 설정

    #layer 정보 프린트 하기
    print(summary(model, input_size=(1, 28, 28)))

    train(epoch, model, train_loader, val_loader)   #학습
    acc, _ = test(model, test_loader)               #테스트

    print('test acc {0:3.5f}' .format(acc))
    writer.close()